# Install Phantom-Omni

The ROS instructions are: http://wiki.ros.org/phantom_omni

The ROS repository is: https://github.com/danepowell/phantom_omni

Development for Phantom-omni: 
- https://fsuarez6.github.io/projects/geomagic-touch-in-ros/
- https://github.com/bharatm11/Geomagic_Touch_ROS_Drivers
    
Follow the instructions to install the devic:omni intall.txt

Additional information: https://github.com/jdrew1303/ros_geomagic_touch_phantom_omni

# Omni control

The final configuration will be:

<img src="./Images/3_omni_nodes.png">

In phantom_omni stack, two diferent packages have to be created:
- omni_moveit: to obtain the omni_pose of pen end-effector
- omni_control: to perform desired conversions and publish the omni_pose_target

In ur5_control package, a python script file has to be created:
- to subscribe the (x,y,z) of the /omni_pose topic for the end-effector position
- perform the desired pose-scaling and safety transformations

In Arduino, the serial_node will:
- subscribe the (R,P,W) orientation
- translate it to servomotor angle rotation
- read the force exerted by the servomotors
- publish forces 

# Endowrist control

The Endowrist module is controlled within Arduino Mega board to interface with:
- Actuators: 4 servomotors
- Sensors: force sensors 

We need to install Rosserial module. Follow the indications to install Arduino and Rosserial module: https://github.com/manelpuig/rubot_hw_ws/blob/master/Documentation/01_Arduino_install.ipynb

Important links are:

http://wiki.ros.org/rosserial_arduino/Tutorials
http://wiki.ros.org/rosserial_arduino/Tutorials/Servo%20Controller

In [ ]:
#if (ARDUINO >= 100)
 #include <Arduino.h>
#else
 #include <WProgram.h>
#endif

#include <Servo.h> 
#include <ros.h>
#include <std_msgs/UInt16.h>

ros::NodeHandle  nh;

Servo servo1;
Servo servo2;

void servo1_cb( const std_msgs::UInt16&  cmd_msg){
  servo1.write(cmd_msg.data); //set servo angle, should be from 0-180  
  digitalWrite(13, HIGH-digitalRead(13));  //toggle led  
}

void servo2_cb( const std_msgs::UInt16&  cmd_msg){
  servo2.write(cmd_msg.data); //set servo angle, should be from 0-180  
  //digitalWrite(13, HIGH-digitalRead(13));  //toggle led  
}
ros::Subscriber<std_msgs::UInt16> sub1("servo1", servo1_cb);
ros::Subscriber<std_msgs::UInt16> sub2("servo2", servo2_cb);

void setup(){
  pinMode(13, OUTPUT);

  nh.initNode();
  nh.subscribe(sub1);
  nh.subscribe(sub2);

  servo1.attach(9); //attach it to pin 9
  servo2.attach(10);//attach it to pin10
}

void loop(){
  nh.spinOnce();
  delay(20);
}

In [ ]:
roscore
rossrun rosserial_python serial_node.py /dev/ttyUSB0

In [ ]:
rostopic pub servo1 std_msgs/UInt16  <angle1> & rostopic pub servo2 std_msgs/UInt16  <angle2>